In [5]:
conda install py-xgboost

Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: done
Solving environment: done

## Package Plan ##

  environment location: /Users/YangsuYun/anaconda3

  added / updated specs:
    - py-xgboost


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    _py-xgboost-mutex-2.0      |            cpu_0           8 KB
    conda-4.11.0               |   py37hecd8cb5_0        14.4 MB
    libxgboost-0.90            |       h0a44026_1         1.2 MB
    py-xgboost-0.90            |   py37h0a44026_1          75 KB
    ------------------------------------------------------------
                                           Total:        15.7 MB

The following NEW packages will be INSTALLED:

  _py-xgboost-mutex  pkgs/main/osx-64::_py-xgboost-mutex-2.0-cpu_0
  libxgboost         pkgs/main/osx-64::libxgboost-0.90-h0a44026_1
  py-xgboost         pkgs/main/

In [5]:
import pandas as pd
import numpy as np

import xgboost as xgb
from tqdm import tqdm
from sklearn.svm import SVC
from keras.models import sequential
from keras.layers.recurrent import LSTM, GRU
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.embeddings import Embedding
#from keras.layers.normalization import BatchNormalization
from tensorflow.keras.layers import BatchNormalization
from keras.utils import np_utils
from sklearn import preprocessing, decomposition, model_selection, metrics, pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from keras.layers import GlobalMaxPooling1D, Conv1D, MaxPooling1D, Flatten, Bidirectional, SpatialDropout1D
from keras.preprocessing import sequence, text
from keras.callbacks import EarlyStopping
from nltk import word_tokenize
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

In [8]:
train = pd.read_csv("../_Kaggle/input/train 3.csv")
test = pd.read_csv("../_Kaggle/input/test.csv")
sample = pd.read_csv("../_Kaggle/input/sample_submission.csv")

In [7]:
train.head()

,id,text,author
0,id26305,"This process, however, afforded me no means of...",EAP
1,id17569,It never once occurred to me that the fumbling...,HPL
2,id11008,"In his left hand was a gold snuff box, from wh...",EAP
3,id27763,How lovely is spring As we looked from Windsor...,MWS
4,id12958,"Finding nothing else, not even gold, the Super...",HPL


In [9]:
test.head()

,id,text
0,id02310,"Still, as I urged our leaving Ireland with suc..."
1,id24541,"If a fire wanted fanning, it could readily be ..."
2,id00134,And when they had broken down the frail door t...
3,id27757,While I was thinking how I should possibly man...
4,id04081,I am not sure to what limit his knowledge may ...


In [10]:
sample.head()

,id,EAP,HPL,MWS
0,id02310,0.403494,0.287808,0.308698
1,id24541,0.403494,0.287808,0.308698
2,id00134,0.403494,0.287808,0.308698
3,id27757,0.403494,0.287808,0.308698
4,id04081,0.403494,0.287808,0.308698


 Multi-class-log-loss as evaluation metric

In [26]:
def multiclass_logloss(actual, predicted, eps = 1e-15):
    # multi class version of Logarithmic Loss metric
    
    #convert 'actual' to a binary array if it's not already:
    if len(actual.shape) == 1:
        actual2 = np.zeros((actual.shape[0], predicted.shape[1]))
        for i, val in enumerate(actual):
            actual2[i, val] =1
            actual = actual2
            
    clip = np.clip(predicted, eps, 1-eps)
    rows = actual.shape[0]
    vsota = np.sum(actual*np.log(clip))
    return -1.0 / rows* vsota

LabelEncoder from scikit-learn to convert text lables to integers 0,1,2

In [13]:
lbl_enc = preprocessing.LabelEncoder()
y = lbl_enc.fit_transform(train.author.values)

In [19]:
xtrain, xvalid, ytrain, yvalid = train_test_split(train.text.values, y,
                                                 stratify=y,
                                                 random_state = 42,
                                                 test_size =0.1,
                                                 shuffle=True)

In [20]:
# 0.9 vs 0.1 로 xtrain vs xvalid 나눔
print(xtrain.shape) 
print(xvalid.shape)

(17621,)
(1958,)


# Build Basic Model : TF-IDF
### - First model is a simple TF-IDF followed by a simple Logistic Regression

TF-IDF(Term Frequency-Inverse Document Frequency)는 단어의 빈도와 역 문서 빈도(문서의 빈도에 특정 식을 취함)를 사용하여 DTM 내의 각 단어들마다 중요한 정도를 가중치로 주는 방법입니다. 우선 DTM을 만든 후, TF-IDF 가중치를 부여합니다.

TF-IDF는 모든 문서에서 자주 등장하는 단어는 중요도가 낮다고 판단하며, 특정 문서에서만 자주 등장하는 단어는 중요도가 높다고 판단합니다. TF-IDF 값이 낮으면 중요도가 낮은 것이며, TF-IDF 값이 크면 중요도가 큰 것입니다

In [23]:
# always start with these features. They work every time
# parameter 
# 1. analyzer : word(home, go, my) or char(a,b,c,d)
# 2. Max_feature: 최대 feature 설정( 행렬의 열/ 테이블의 컬럼)
# 3. min_df : 특정단어가 나타나는 문서의 수 (예: home 단어가 포함된 문서의 수는 3개 df=3)
# 4. n-gram : 단어의 묶음  (ex:(1,2)단어의 묶음을 1개부터 2개까지 설정 ex go home, very good
tfv = TfidfVectorizer(min_df=3, max_features= None, strip_accents = 'unicode',
                    analyzer='word', token_pattern=r'\w{1,}', 
                    ngram_range=(1,3), use_idf =1, smooth_idf=1, sublinear_tf = 1,
                    stop_words ='english')

# fitting TF-idf to both training and test set
tfv.fit(list(xtrain) + list(xvalid))
xtrain_tfv = tfv.transform(xtrain)
xvalid_tfv = tfv.transform(xvalid)

In [40]:
print(xtrain_tfv)

  (0, 11813)	0.2469424385443994
  (0, 10050)	0.31534173991151543
  (0, 7734)	0.2621357570531394
  (0, 6078)	0.21913481148346806
  (0, 5920)	0.2699803359499066
  (0, 5641)	0.28210670387497144
  (0, 3647)	0.33928338965309934
  (0, 3312)	0.2913532806410929
  (0, 2813)	0.3345950093082599
  (0, 2161)	0.3474674493892856
  (0, 1489)	0.3710563608683541
  (1, 14862)	0.3467364483300827
  (1, 11446)	0.23785570257025115
  (1, 9156)	0.3385709613222204
  (1, 8899)	0.49640563902064305
  (1, 8268)	0.4364136618558541
  (1, 4746)	0.3259827441631435
  (1, 72)	0.4067284254948717
  (2, 14071)	0.3240192548729904
  (2, 13042)	0.25915485523649134
  (2, 12014)	0.26382082678856383
  (2, 11546)	0.16409370088652775
  (2, 9656)	0.25408177755421213
  (2, 9422)	0.26141001709560185
  (2, 7995)	0.28804138618156144
  :	:
  (17617, 2312)	0.3765818334354421
  (17618, 14413)	0.41062696740663335
  (17618, 13333)	0.41457868692900307
  (17618, 12513)	0.2857797208393344
  (17618, 8238)	0.3745429502598771
  (17618, 3074)	0.384

In [27]:
# fitting a simple Logistic Regression on TFIDF
clf = LogisticReg`ression(C=1.0)
clf.fit(xtrain_tfv, ytrain)
predictions = clf.predict_proba(xvalid_tfv)

print("logloss: %0.3f" % multiclass_logloss(yvalid, predictions))

logloss: 0.572


/Users/YangsuYun/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


## Using Count vectorizer

In [35]:
ctv = CountVectorizer(analyzer ='word', token_pattern=r'\w{1,}',
                     ngram_range=(1,3), stop_words='english')

#Fitting Count Vectorizer to both Training and Test sets
ctv.fit(list(xtrain) + list(xvalid))
xtrain_ctv = ctv.transform(xtrain)
xvalid_ctv = ctv.transform(xvalid)

In [36]:
#fitting a simple Logistic regression on count
clf = LogisticRegression(C=1.0)
clf.fit(xtrain_ctv, ytrain)
predictions = clf.predict_proba(xvalid_ctv)
print("logloss: %0.3f" % multiclass_logloss(yvalid, predictions))

logloss: 0.527


/Users/YangsuYun/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


## Navies Bayes

In [38]:
# fitting a simple Naive bayes on TF-IDF
# navies bayes + TF-IDF
clf = MultinomialNB()
clf.fit(xtrain_tfv, ytrain)
predictions = clf.predict_proba(xvalid_tfv)
print("logloss: %0.3f" % multiclass_logloss(yvalid, predictions))

logloss: 0.578


In [43]:
# navies bayes + count

clf = MultinomialNB()
clf.fit(xtrain_ctv, ytrain)
predictions = clf.predict_proba(xvalid_ctv)
print("logloss: %0.3f" % multiclass_logloss(yvalid, predictions))

logloss: 0.485


# SVM

1. Reduce the number of features from the TF-IDF using singular value decomposition

2. standardize the data

In [42]:
# Apply SVD, i chose 120 component. 120-2-- components are good enough for SVM model
# reduce the number of feature
svd = decomposition.TruncatedSVD(n_components = 120) #decrease the feature
svd.fit(xtrain_tfv)
xtrain_svd = svd.transform(xtrain_tfv)
xvalid_svd = svd.transform(xvalid_tfv)

In [47]:
#scale the daa obtained from svd. renaimg variable to reuse without scaling 
#sclaeling
scl = preprocessing.StandardScaler()
scl.fit(xtrain_svd)
scl.fit(xvalid_svd)

xtrain_svd_scl = scl.transform(xtrain_svd)
xvalid_svd_scl = scl.transform(xvalid_svd)

In [48]:
# Fitting a simple SVM
clf = SVC(C=1.0, probability = True) # since we need probability
clf.fit(xtrain_svd_scl, ytrain)
print("logloss %0.3f" %multiclass_logloss(yvalid, predictions))

KeyboardInterrupt: 

## Xgboost

In [49]:
# fitting a simple xgboost on tf-idf

clf = xgb.XGBClassifier(max_depth = 5, n_estimators = 200, colsample_bytree = 0.8,
                       subsample =0.8, nthred =10, learning_rate = 0.1)
clf.fit(xtrain_tfv.tocsc(), ytrain)
predictions = clf.predict_proba(xvalid_tfv.tocsc())

print("logloss: %0.3f" % multiclass_logloss(yvalid, predictions))

logloss: 0.815


In [50]:
#fitting a imple xgboost on countword

clf = xgb.XGBClassifier(max_depth = 7, n_estimators = 200, colsample_bytree = 0.8,
                       subsample =0.8, nthread = 10, learning_rate = 0.1)
clf.fit(xtrain_ctv.tocsc(), ytrain)
predictions = clf.predict_proba(xvalid_ctv.tocsc())
print("logloss : %0.3f" % multiclass_logloss(yvalid, predictions))

logloss : 0.772


# Grid Search

In [51]:
#scoring function
mll_scorer = metrics.make_scorer(multiclass_logloss, greater_is_better = False, needs_proba = True)

In [52]:
# piple line

# Initialize SVD
svd = TruncatedSVD()

#Initialize the stndard scaler
scl = preprocessing.StandardScaler()

# we will use logistic regression here..
lr_model = LogisticRegression()

#create the pipleline
clf = pipeline.Pipeline([('svd', svd),
                        ('scl',scl),
                        ('lr',lr_model)])

In [53]:
# gridsearch parameter

param_grid={'svd__n_components':[120,180],
           'lr__C':[0.1, 1.0, 10],
           'lr__penalty':['l1','l2']}

In [61]:
# Initialize Grid Search Model
model = GridSearchCV(estimator=clf, param_grid=param_grid, scoring=mll_scorer,
                                 verbose=10, n_jobs=-1,  refit=True, cv=2)

# Fit Grid Search Model
model.fit(xtrain_tfv, ytrain)  # we can use the full data here but im only using xtrain
print("Best score: %0.3f" % model.best_score_)
print("Best parameters set:")
best_parameters = model.best_estimator_.get_params()
for param_name in sorted(param_grid.keys()):
    print("\t%s: %r" % (param_name, best_parameters[param_name]))

Fitting 2 folds for each of 12 candidates, totalling 24 fits


/Users/YangsuYun/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:372: FitFailedWarning: 
12 fits failed out of a total of 24.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
12 fits failed with the following error:
Traceback (most recent call last):
  File "/Users/YangsuYun/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_validation.py", line 681, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/YangsuYun/anaconda3/lib/python3.7/site-packages/sklearn/pipeline.py", line 394, in fit
    self._final_estimator.fit(Xt, y, **fit_params_last_step)
  File "/Users/YangsuYun/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py", line 1461, in fit
    so

Best score: -0.738
Best parameters set:
	lr__C: 1.0
	lr__penalty: 'l2'
	svd__n_components: 180


In [62]:
# multinomial naive
nb_model = MultinomialNB()

#Create the pipeline
clf = pipeline.Pipeline([('nb', nb_model)])

parameter_grid = {'nb__alpha':[0.001, 0.01, 0.1, 1, 10, 100]}

#Initialize Grid Search Model
model = GridSearchCV(estimator = clf, param_grid = param_grid, scoring = mll_scorer, 
                    verbose = 10, n_jobs = -1, refit = True, cv =2 )

In [64]:
# fit grid search Model
model.fit(xtrain_tfv, ytrain)
print("Best score : % 0.3f" % model.best_score_)
print("Best parameters set:")
best_parameters = model.best_estimator_.get_params()
for param_name in sorted(param_grid.keys()):
    print("\t%s: %r" %(param_name, best_parameters[param_name]))

Fitting 2 folds for each of 12 candidates, totalling 24 fits


ValueError: Invalid parameter lr for estimator Pipeline(steps=[('nb', MultinomialNB())]). Check the list of available parameters with `estimator.get_params().keys()`.

# Word Vectors
GloVe vectors, word2vec and fasttext

In [1]:
exampe= "Family is not an important thing. It's everything"
stop_words = set(stopwords.words('english'))

word_tokens = word_tokenize(example)
word_tokens

NameError: name 'stopwords' is not defined

In [2]:
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords


In [3]:
raw_text = "A barber is a person. a barber is good person. a barber is huge person. he Knew A Secret! The Secret He Kept is huge secret. Huge secret. His barber kept his word. a barber kept his word. His barber kept his secret. But keeping and keeping such a huge secret to himself was driving the barber crazy. the barber went up a huge mountain."

In [6]:
sentences = sent_tokenize(raw_text)
print(sentences)

['A barber is a person.', 'a barber is good person.', 'a barber is huge person.', 'he Knew A Secret!', 'The Secret He Kept is huge secret.', 'Huge secret.', 'His barber kept his word.', 'a barber kept his word.', 'His barber kept his secret.', 'But keeping and keeping such a huge secret to himself was driving the barber crazy.', 'the barber went up a huge mountain.']


In [11]:
# 텍스트가 -> 문장단위로 토큰화 
vocab ={}
preprocessed_sentences =[]
stop_words = set(stopwords.words('english'))
for sentence in sentences:
    tokenized_sentence = word_tokenize(sentence)
    result =[]
    
    for word in tokenized_sentence:
        word = word.lower()
        if word not in stop_words:
            if len(word) > 2:
                result.append(word)
                if word not in vocab:
                    vocab[word] = 0
                vocab[word] += 1
    preprocessed_sentences.append(result)
print(preprocessed_sentences)

[['barber', 'person'], ['barber', 'good', 'person'], ['barber', 'huge', 'person'], ['knew', 'secret'], ['secret', 'kept', 'huge', 'secret'], ['huge', 'secret'], ['barber', 'kept', 'word'], ['barber', 'kept', 'word'], ['barber', 'kept', 'secret'], ['keeping', 'keeping', 'huge', 'secret', 'driving', 'barber', 'crazy'], ['barber', 'went', 'huge', 'mountain']]


In [12]:
print(vocab)

{'barber': 8, 'person': 3, 'good': 1, 'huge': 5, 'knew': 1, 'secret': 6, 'kept': 4, 'word': 2, 'keeping': 2, 'driving': 1, 'crazy': 1, 'went': 1, 'mountain': 1}


# Word Vectors

In [1]:
#load the GloVe vectors in a dictionary:
embeddings_index = {}
f = open('../input/glove840b300dtxt/glove.840B.300d.txt')

for line in tqdm(f):
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtye='flost32')
    embeddings_index[word] =coefs
f.clsoe()

print('Found %s word vectors.' %len(embeddings_index))

FileNotFoundError: [Errno 2] No such file or directory: '../input/glove840b300dtxt/glove.840B.300d.txt'